# Projeto – Maximização de Receita | EDA e Pré-processamento

Este notebook dá continuidade à análise após os testes de **Shapiro-Wilk** e **Levene**, aprofundando a EDA e estruturando o pré-processamento para modelagem de Regressão Linear.

In [ ]:
# Importações e configuração
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import KFold, TimeSeriesSplit, cross_val_score, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from src.modeling.train_pipeline import load_dataset

pd.set_option('display.max_columns', 100)
pd.options.display.float_format = '{:.4f}'.format
sns.set(style='whitegrid')
np.random.seed(42)

print('Ambiente configurado.')

In [ ]:
# Carregamento e inspeção inicial (usa fallback sintético quando necessário)
df = load_dataset()
df.head()

In [ ]:
# Informações gerais
df.info()
df.describe().T

## Limpeza inicial
- Remoção de duplicatas.
- Conversão de coluna de data (se necessário).
- Criação de variáveis auxiliares de receita.

In [ ]:
# Remoção de duplicatas
df = df.drop_duplicates()

# Conversão de data (se existir a coluna 'data')
if 'data' in df.columns:
    df['data'] = pd.to_datetime(df['data'], errors='coerce')

# Criação de variáveis de receita
if {'preco_final','quantidade_vendida_dia'}.issubset(df.columns):
    df['receita_dia'] = df['preco_final'] * df['quantidade_vendida_dia']
if {'preco_final','quantidade_vendida_mes'}.issubset(df.columns):
    df['receita_mes'] = df['preco_final'] * df['quantidade_vendida_mes']

df.head()

## EDA – Distribuições e correlações
- Histograma e boxplot para variáveis numéricas.
- Matriz de correlação.
- Relação entre `desconto_pct` e `quantidade_vendida_mes`.

In [ ]:
# Variáveis numéricas
num_cols = df.select_dtypes(include=['number']).columns.tolist()
fig, axes = plt.subplots(len(num_cols), 2, figsize=(10, 4*len(num_cols)))
for i, col in enumerate(num_cols):
    sns.histplot(df[col], kde=True, ax=axes[i,0])
    axes[i,0].set_title(f'Histograma – {col}')
    sns.boxplot(x=df[col], ax=axes[i,1])
    axes[i,1].set_title(f'Boxplot – {col}')
plt.tight_layout()
plt.show()

# Correlação
corr = df[num_cols].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', annot=True, fmt='.2f')
plt.title('Matriz de Correlação')
plt.show()

# Relação desconto vs quantidade vendida (mensal)
if {'desconto_pct','quantidade_vendida_mes'}.issubset(df.columns):
    plt.figure(figsize=(8,5))
    sns.scatterplot(x='desconto_pct', y='quantidade_vendida_mes', data=df, alpha=0.6)
    plt.title('Desconto (%) vs Quantidade Vendida (mês)')
    plt.show()

## Tratamento de outliers (winsorização por IQR)
Aplicamos capping para reduzir a influência de outliers severos nas variáveis numéricas.

In [ ]:
def winsorize_iqr(series, k=1.5):
    q1 = series.quantile(0.25)
    q3 = series.quantile(0.75)
    iqr = q3 - q1
    lower = q1 - k*iqr
    upper = q3 + k*iqr
    return series.clip(lower, upper)

df_w = df.copy()
for col in num_cols:
    df_w[col] = winsorize_iqr(df_w[col])

df_w.describe().T

## Seleção de features e alvo
Como objetivo de negócio, modelaremos `quantidade_vendida_mes` e, posteriormente, maximizaremos `receita_mes = preco_final * quantidade_vendida_mes`.

In [ ]:
target = 'quantidade_vendida_mes' if 'quantidade_vendida_mes' in df_w.columns else 'quantidade_vendida_dia'
feature_cols = [c for c in ['custo_producao','preco_original','desconto_pct','preco_final'] if c in df_w.columns]
X = df_w[feature_cols].copy()
y = df_w[target].copy()
X.head(), y.head()

## Validação cruzada (CV)
A avaliação será feita por validação cruzada (5 folds). Para séries temporais, usamos TimeSeriesSplit.

In [ ]:
# Seleção do esquema de CV
cv = TimeSeriesSplit(n_splits=5) if ('data' in df_w.columns and pd.api.types.is_datetime64_any_dtype(df_w['data'])) else KFold(n_splits=5, shuffle=True, random_state=42)

# Pipeline simples (dados sem normalização); ajuste se necessário
model = Pipeline([('model', LinearRegression())])

# Métricas CV
scores_rmse = -cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error')
scores_mae = -cross_val_score(model, X, y, cv=cv, scoring='neg_mean_absolute_error')
scores_r2 = cross_val_score(model, X, y, cv=cv, scoring='r2')
print(f'CV RMSE: {scores_rmse.mean():.4f} ± {scores_rmse.std():.4f}')
print(f'CV MAE:  {scores_mae.mean():.4f} ± {scores_mae.std():.4f}')
print(f'CV R2:   {scores_r2.mean():.4f} ± {scores_r2.std():.4f}')

# Predições OOF para diagnóstico
y_pred_oof = cross_val_predict(model, X, y, cv=cv)
plt.figure(figsize=(6,6))
plt.scatter(y, y_pred_oof, alpha=0.5)
plt.xlabel('y real')
plt.ylabel('y predito (OOF)')
plt.title('Diagnóstico OOF – LinearRegression')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--')
plt.tight_layout()
plt.show()

## Próximos passos
- Construir pipeline de regressão linear sem normalização (dados não normais).
- Avaliar com métricas (RMSE, R²) e validação cruzada.
- Exportar coeficientes e intercepto para uso no front-end (HTML/JS).